In [81]:
from google.transit import gtfs_realtime_pb2
import requests
import plotly.express as px
import pandas as pd
import json
import time

In [85]:
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://www.rtd-denver.com/files/gtfs-rt/VehiclePosition.pb')
feed.ParseFromString(response.content)

latitudes = []
longitudes = []
ids = []

for entity in feed.entity:
    latitudes.append(entity.vehicle.position.latitude)
    longitudes.append(entity.vehicle.position.longitude)
    ids.append(entity.vehicle.vehicle.label)

df = pd.DataFrame({'label': ids, 'lat': latitudes, 'lon': longitudes})

px.scatter_mapbox(df, lat='lat', lon='lon', height=500, zoom=9, mapbox_style='carto-positron', hover_name='label')

In [82]:
def extract_entity_data(entity):
    return {
        'insert_ts': int(time.time()),
        'vehicle_update_ts': entity.vehicle.timestamp,
        'lat': entity.vehicle.position.latitude,
        'lon': entity.vehicle.position.longitude,
        'trip': entity.vehicle.trip.trip_id,
        'route_id': entity.vehicle.trip.route_id,
        'stop_id': entity.vehicle.stop_id,
        'vehicle_id': entity.vehicle.vehicle.id,
        'vehicle_label': entity.vehicle.vehicle.label
    }

In [83]:
to_export = []
for entity in feed.entity:
    to_export.append(extract_entity_data(entity))

In [84]:
with open('rtd_vehicle_positions.json', 'a') as f:
    json.dump(to_export, f)

In [46]:
routes = pd.read_csv('routes.txt')
stops = pd.read_csv('stops.txt')
trips = pd.read_csv('trips.txt')

In [86]:
routes[routes.route_id == 'A']

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
97,A,RTD,A,Union Station to Denver Airport Station,This Route Travels Eastbound & Westbound,2,http://www.rtd-denver.com/Schedules.shtml,57C1E9,FFFFFF


In [63]:
stops[stops.stop_id == 34667]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
5667,34667,34667,Union Station Track 1,Vehicles Travelling West,39.753698,-105.0002,A,NaN,0,33727.0,NaN,1


In [62]:
trips[trips.trip_id == 114913008]

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
